In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [2]:
# TAG Word
PAD = "<PADDING>" 
STA = "<START>"  
END = "<END>" 
OOV = "<OOV>"

# Tag Index
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# Data Type
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# MAXiMUM Sequnce
max_sequences = 30

# Embedding Vector Dimension
embedding_dim = 100

# LSTM Hidden Layer Dimension
lstm_hidden_dim = 128

# Filter
RE_FILTER = re.compile("[.,!?\"':;~()]")

# Chatbot Data Load
chatbot_data = pd.read_csv('smuChattinghaejoTraining.txt', names=['Q', 'A'], sep='\t')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [3]:
# Question Data Size = Answer Data Size
len(question)


18442

In [4]:
# 랜덤한 챗봇 데이터 출력 (10개)
ramdomNum = np.random.randint(0, len(question), size=10)
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 전략평가팀 사무실 위치좀
A : office 전략평가팀 위치

Q : 화공신소재과 번호가 머임
A : office 화공신소재전공 연락처

Q : 지적재산권과 사무실 위치좀
A : office 지적재산권전공 위치

Q : 양동국 교수님 연락처 알려줘
A : professor 연락처

Q : 국안과 과사 번호가 뭐야?
A : office 국가안보학과 연락처

Q : 금융학부 장소가 어딘가요?
A : office 경제금융학부 위치

Q : 지적재산권과 어케 가요 ?사무실 위치
A : office 지적재산권전공 위치

Q : 영양 과사 위치가 어떻게 되나요 ?
A : office 식품영양학전공 위치

Q : 디자인센터 위치좀
A : office 상명아트센터 위치

Q : 기획 전화번호
A : office 기획예산팀 연락처



In [5]:
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [6]:
# 형태소분석 수행
question = pos_tag(question)
# Question 만 형태소 분석 실시 (Answer는 이미 정해진 타입이 있기 때문)

# 변경된 점 출력
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 전략 평가 팀 사무실 위치 좀
A : office 전략평가팀 위치

Q : 화공 신소재 과 번호 가 머임
A : office 화공신소재전공 연락처

Q : 지적재산권 과 사무실 위치 좀
A : office 지적재산권전공 위치

Q : 양동국 교수 님 연락처 알려줘
A : professor 연락처

Q : 국 안과 과사 번호 가 뭐 야
A : office 국가안보학과 연락처

Q : 금융 학부 장소 가 어딘가 요
A : office 경제금융학부 위치

Q : 지적재산권 과 어케 가요 사무실 위치
A : office 지적재산권전공 위치

Q : 영양 과사 위치 가 어떻게 되나요
A : office 식품영양학전공 위치

Q : 디자인 센터 위치 좀
A : office 상명아트센터 위치

Q : 기획 전화번호
A : office 기획예산팀 연락처



In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words = sorted(list(words))
words[:0] = [PAD, STA, END, OOV]



In [8]:
# 단어 개수
len(words)


440

In [9]:
# 단어 출력
words


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 'IOT',
 'iot',
 'office',
 'professor',
 '가',
 '가나',
 '가는',
 '가복',
 '가야해',
 '가요',
 '가족',
 '가족복지학과',
 '갈',
 '감',
 '강',
 '강성',
 '개발',
 '건강',
 '게임',
 '게임전공',
 '게임학',
 '겜',
 '경',
 '경과',
 '경영',
 '경영학',
 '경영학부',
 '경제',
 '경제금융학부',
 '경제학',
 '계',
 '계셔',
 '계신지',
 '계심',
 '고',
 '공',
 '공간',
 '공간환경학부',
 '공과',
 '공임',
 '공학',
 '공학교육혁신센터',
 '공환',
 '과',
 '과사',
 '관리',
 '관리팀',
 '교수',
 '교원',
 '교원인사',
 '교원인사팀',
 '교육',
 '교육학',
 '교육학과',
 '교직',
 '교직지원센터',
 '교학',
 '교학팀',
 '국',
 '국가',
 '국가안보학과',
 '국교',
 '국안',
 '국어',
 '국어교육',
 '국어교육과',
 '국제',
 '국제언어문화교육원',
 '국제학생지원팀',
 '권',
 '귀',
 '귀지',
 '글',
 '글경',
 '글로벌',
 '글로벌경영학과',
 '글로벌랭귀지센터',
 '금',
 '금융',
 '기기',
 '기초',
 '기초교육센터',
 '기획',
 '기획예산팀',
 '김',
 '김경일',
 '김동아',
 '김미숙',
 '김유천',
 '김정임',
 '김지영',
 '김한식',
 '까요',
 '께',
 '나',
 '능력',
 '님',
 '단',
 '담소',
 '대대',
 '대체',
 '대학',
 '대학일자리센터',
 '도대체',
 '돼',
 '되',
 '되나요',
 '됩니까',
 '디자인',
 '란',
 '랭',
 '로',
 '를',
 '만나고',
 '만나려면',
 '머',
 '머임',
 '메이저',
 '메일',
 '멜',
 '면',
 '명',
 '무슨',
 '무예',
 '

In [10]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [11]:
np.save("word_to_index",word_to_index)
np.save("index_to_word",index_to_word)

In [12]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items()))


{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 'IOT': 4,
 'iot': 5,
 'office': 6,
 'professor': 7,
 '가': 8,
 '가나': 9,
 '가는': 10,
 '가복': 11,
 '가야해': 12,
 '가요': 13,
 '가족': 14,
 '가족복지학과': 15,
 '갈': 16,
 '감': 17,
 '강': 18,
 '강성': 19,
 '개발': 20,
 '건강': 21,
 '게임': 22,
 '게임전공': 23,
 '게임학': 24,
 '겜': 25,
 '경': 26,
 '경과': 27,
 '경영': 28,
 '경영학': 29,
 '경영학부': 30,
 '경제': 31,
 '경제금융학부': 32,
 '경제학': 33,
 '계': 34,
 '계셔': 35,
 '계신지': 36,
 '계심': 37,
 '고': 38,
 '공': 39,
 '공간': 40,
 '공간환경학부': 41,
 '공과': 42,
 '공임': 43,
 '공학': 44,
 '공학교육혁신센터': 45,
 '공환': 46,
 '과': 47,
 '과사': 48,
 '관리': 49,
 '관리팀': 50,
 '교수': 51,
 '교원': 52,
 '교원인사': 53,
 '교원인사팀': 54,
 '교육': 55,
 '교육학': 56,
 '교육학과': 57,
 '교직': 58,
 '교직지원센터': 59,
 '교학': 60,
 '교학팀': 61,
 '국': 62,
 '국가': 63,
 '국가안보학과': 64,
 '국교': 65,
 '국안': 66,
 '국어': 67,
 '국어교육': 68,
 '국어교육과': 69,
 '국제': 70,
 '국제언어문화교육원': 71,
 '국제학생지원팀': 72,
 '권': 73,
 '귀': 74,
 '귀지': 75,
 '글': 76,
 '글경': 77,
 '글로벌': 78,
 '글로벌경영학과': 79,
 '글로벌랭귀지센터': 80,
 '금': 81,
 '금융': 82,
 '기기': 

In [13]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items()))


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: 'IOT',
 5: 'iot',
 6: 'office',
 7: 'professor',
 8: '가',
 9: '가나',
 10: '가는',
 11: '가복',
 12: '가야해',
 13: '가요',
 14: '가족',
 15: '가족복지학과',
 16: '갈',
 17: '감',
 18: '강',
 19: '강성',
 20: '개발',
 21: '건강',
 22: '게임',
 23: '게임전공',
 24: '게임학',
 25: '겜',
 26: '경',
 27: '경과',
 28: '경영',
 29: '경영학',
 30: '경영학부',
 31: '경제',
 32: '경제금융학부',
 33: '경제학',
 34: '계',
 35: '계셔',
 36: '계신지',
 37: '계심',
 38: '고',
 39: '공',
 40: '공간',
 41: '공간환경학부',
 42: '공과',
 43: '공임',
 44: '공학',
 45: '공학교육혁신센터',
 46: '공환',
 47: '과',
 48: '과사',
 49: '관리',
 50: '관리팀',
 51: '교수',
 52: '교원',
 53: '교원인사',
 54: '교원인사팀',
 55: '교육',
 56: '교육학',
 57: '교육학과',
 58: '교직',
 59: '교직지원센터',
 60: '교학',
 61: '교학팀',
 62: '국',
 63: '국가',
 64: '국가안보학과',
 65: '국교',
 66: '국안',
 67: '국어',
 68: '국어교육',
 69: '국어교육과',
 70: '국제',
 71: '국제언어문화교육원',
 72: '국제학생지원팀',
 73: '권',
 74: '귀',
 75: '귀지',
 76: '글',
 77: '글경',
 78: '글로벌',
 79: '글로벌경영학과',
 80: '글로벌랭귀지센터',
 81: '금',
 82: '금융',
 83: '기

In [14]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [15]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력
x_encoder[0]


array([434, 131,  51, 100, 122, 291, 139, 231,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [16]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력
x_decoder[0]


array([  1,   7, 292,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [17]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력
y_decoder[0]


array([  7, 292,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [18]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [19]:
# ---------- MARK:- Model Encoder

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



# ---------- MARK:- Model Decoder

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# ---------- MARK:- Model Setting

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

In [20]:
# ---------- MARK:- Model Encoder

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



# ---------- MARK:- Predict Model Decoder

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [21]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

# Model.fit

In [22]:
# 모델 교육
model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=10,
                        batch_size=64,
                        verbose=1)
# 나중에 챗봇으로 쓸 때 epoch랑 batch_size 수치 잘 수정하기
# 추후 빡세게 Model.fit 하고 Model 저장 후 Load해서 하기

Epoch 1/10
289/289 [==============================] - 48s 78ms/step - loss: 0.7349 - acc: 0.9014
Epoch 2/10
289/289 [==============================] - 24s 84ms/step - loss: 0.1216 - acc: 0.9691
Epoch 3/10
289/289 [==============================] - 24s 83ms/step - loss: 0.1008 - acc: 0.9723
Epoch 4/10
289/289 [==============================] - 26s 90ms/step - loss: 0.0861 - acc: 0.9744
Epoch 5/10
289/289 [==============================] - 26s 89ms/step - loss: 0.0734 - acc: 0.9765
Epoch 6/10
289/289 [==============================] - 25s 85ms/step - loss: 0.0628 - acc: 0.9790
Epoch 7/10
289/289 [==============================] - 27s 93ms/step - loss: 0.0505 - acc: 0.9825
Epoch 8/10
289/289 [==============================] - 25s 86ms/step - loss: 0.0405 - acc: 0.9858
Epoch 9/10
289/289 [==============================] - 24s 83ms/step - loss: 0.0298 - acc: 0.9905
Epoch 10/10
289/289 [==============================] - 24s 85ms/step - loss: 0.0209 - acc: 0.9934


In [23]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [24]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)
    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    
    print(indexs)
    return sentence

In [25]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 번호가 뭐야')
input_seq


array([[ 51, 100, 152,   8, 139, 231,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [26]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[7, 252, 2]


'professor 연락처 '

In [27]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 위치가 뭐야?')
input_seq


array([[ 51, 100, 272,   8, 139, 231,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [28]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[7, 272, 2]


'professor 위치 '

In [29]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('컴퓨터과학과 위치가 어디야')
input_seq


array([[384,  47, 272,   8, 233, 231,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [30]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[6, 385, 272, 2]


'office 컴퓨터과학과 위치 '

In [31]:
# generate_text(make_predict_input(input()))

#### Kakao-brain의 Pororo 사용 : Text에서 PERSON을 찾을 수 있음 -> Professor 명령일 경우 PERSON을 추출하여 Professor 명령에 PERSON을 넣어줌
from pororo import Pororo
ner = Pororo(task="ner",lang='ko')


sentence = input()
pf = 'professor' in generate_text(make_predict_input(sentence))
if pf:
    for name in ner(sentence):
        if 'PERSON' in name:
            orderText = generate_text(make_predict_input(sentence)).split(" ")
            print(orderText[0] + " " +name[0]+ " "+orderText[1] )
else:
    print(generate_text(make_predict_input(sentence)))
    
#### EX) 홍길동 교수님 연락처 알려줘 -> professor 연락처 + 홍길동 -> professor 홍길동 연락처

# Model Save

In [32]:
from keras.models import load_model

model.save('chatbot_model.h5')

# Model Load

from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('encoder_model.json', 'encoder_model_weights.h5')
decoder = load_model('decoder_model.json', 'decoder_model_weights.h5')

from keras.models import load_model
model = load_model('chatbot_model.h5')

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    44000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    44000       input_2[0][0]                    
______________________________________________________________________________________________